# dnspython Resolver

The `socket` module of the Python standard library provides basic functions for resolving hostnames (`gethostbyname` and `gethostbyname_ex`) as implemented by the C library. The resolver of the [dnspython](http://www.dnspython.org/) allows to bypass the C library and query DNS servers directly.
```
pip install dnspython
```

## Constants
Before we start querying DNS servers it is useful to know about some constants and convenience functions [dnspython](http://www.dnspython.org/) provides. The `dns.rdataclass` module defines the DNS data clasess out of which only Internet `IN` is relevant (or has anyone ever seen `CHAOS` or `HESIOD` implemented???). 

In [4]:
import dns.rdataclass
dns.rdataclass.IN

1

The constants of the `dns.rdatatype` module are a lot more important as they list the types of records commonly 
served by DNS servers.

In [5]:
import dns.rdatatype
dns.rdatatype.A, dns.rdatatype.MX, dns.rdatatype.PTR, dns.rdatatype.SOA, dns.rdatatype.TXT

(1, 15, 12, 6, 16)

## Using the default resolvers
On Unix systems the file `/etc/resolv.conf` may list (on Linux up to three) nameservers that the C library functions shall query for resolving host names that are not found in `/etc/hosts` or other local databases like LDAP or NIS as defined in `/etc/nsswitch.conf`.

Unless one requires exact control which DNS server(s) to query, the `dns.resolver.query()` function (actually a wrapper for the more versatile `dns.resolver.Resolver.query()` method) is the right choice. It will simply use the servers and other settings configured in `/etc/resolv.conf`. The function `dns.resolver.get_default_resolver()` returns the details. Check the `resolv.conf(5)` manual page for more information.

In [6]:
import dns.resolver
dns.resolver.get_default_resolver().nameservers

['10.1.2.1', '8.8.8.8', '8.8.4.4']

In [7]:
dns.resolver.get_default_resolver().domain

<DNS name juenemann.local.>

In [8]:
dns.resolver.get_default_resolver().search

[<DNS name juenemann.local.>, <DNS name juenemann.net.>]

In [9]:
dns.resolver.get_default_resolver().timeout

2.0

### Querying A records
The `dns.resolver.query()` function accepts a number of arguments but in most cases only the first one (`qname`) or two (`rdtype`) are required. Note how the returned value for an `A` query (the default) contains much more information than just the IP address. 

In [10]:
answers = dns.resolver.query('www.google.com')
answers

In [11]:
answers.canonical_name.to_text(), answers.canonical_name.to_unicode()

('www.google.com.', u'www.google.com.')

In [12]:
answers.expiration

1492550783.903062

In [13]:
answers.qname

<DNS name www.google.com.>

In [14]:
answers.qname.to_text(), answers.qname.to_unicode()

('www.google.com.', u'www.google.com.')

In [15]:
answers.rdclass == dns.rdataclass.IN

True

In [16]:
answers.rdtype == dns.rdatatype.A

True

In [17]:
answers.response

<DNS message, ID 37526>

In [18]:
answers.response.edns

-1

In [19]:
answers.response.time

0.023239850997924805

In [20]:
answers.response.flags       # https://tools.ietf.org/html/rfc1035 4.1.1. Header section format   

33152

In [21]:
answers.response.flags & 0b1000000000000000          # 0=query, 1=response

32768

In [22]:
answers.response.flags & 0b0000010000000000          # 1=authoratative

0

In [23]:
answers.response.flags & 0b0000001000000000          # 1=truncated

0

In [24]:
answers.response.flags & 0b0000000100000000          # 1=recursive desired (copied into response)

256

In [25]:
answers.response.flags & 0b0000000010000000          # 1=recursion available

128

In [26]:
answers.response.rcode()                             # errors?

0

In [27]:
answers.rrset

<DNS www.google.com. IN A RRset>

In [28]:
len(answers.rrset)

15

In [29]:
list(answers.rrset)

[<DNS IN A rdata: 150.101.161.237>,
 <DNS IN A rdata: 150.101.161.215>,
 <DNS IN A rdata: 150.101.161.221>,
 <DNS IN A rdata: 150.101.161.219>,
 <DNS IN A rdata: 150.101.161.249>,
 <DNS IN A rdata: 150.101.161.234>,
 <DNS IN A rdata: 150.101.161.226>,
 <DNS IN A rdata: 150.101.161.251>,
 <DNS IN A rdata: 150.101.161.245>,
 <DNS IN A rdata: 150.101.161.222>,
 <DNS IN A rdata: 150.101.161.211>,
 <DNS IN A rdata: 150.101.161.236>,
 <DNS IN A rdata: 150.101.161.230>,
 <DNS IN A rdata: 150.101.161.241>,
 <DNS IN A rdata: 150.101.161.207>]

In [30]:
answers.rrset[0].address

u'150.101.161.237'

In [31]:
answers.rrset[0].rdclass == dns.rdataclass.IN

True

In [32]:
answers.rrset[0].rdclass == dns.rdatatype.A

True

### Querying other record types
Of course one query other records than `A` type. The second argument to `dns.resolver.query` accepts the contants defined in `dns.rdatatype` or simply a string value. The attributes of the returned records are specific ot the queried type, e.g. `MX` records have a `preference` attribute.

#### Example: Mail Exchanger  (MX) records

In [33]:
answers = dns.resolver.query('google.com', 'MX')
len(answers.rrset)

5

In [34]:
answers.rrset[0]

<DNS IN MX rdata: 50 alt4.aspmx.l.google.com.>

In [35]:
answers.rrset[0].exchange.to_text()

'alt4.aspmx.l.google.com.'

In [36]:
answers.rrset[0].preference

50

In [37]:
answers.rrset[0].rdtype == dns.rdatatype.MX

True

#### Example: Start of Authority (SOA) record

In [38]:
answers = dns.resolver.query('google.com', 'SOA')
len(answers.rrset)

1

In [39]:
answers.rrset[0]

<DNS IN SOA rdata: ns2.google.com. dns-admin.google.com. 153472704 900 900 1800 60>

In [40]:
answers.rrset[0].mname.to_text()

'ns2.google.com.'

In [41]:
answers.rrset[0].serial

153472704

In [42]:
answers.rrset[0].refresh

900

#### Example: PTR record

In [44]:
answers = dns.resolver.query('4.4.8.8.in-addr.arpa', 'PTR')
len(answers)

1

In [45]:
answers.rrset[0]

<DNS IN PTR rdata: google-public-dns-b.google.com.>

In [51]:
answers.rrset[0].to_text()

'google-public-dns-b.google.com.'

#### Example SRV record

In [54]:
answers = dns.resolver.query('_http._tcp.juenemann.net', 'SRV')
len(answers)

1

In [55]:
answers.rrset[0]

<DNS IN SRV rdata: 10 50 80 www.juenemann.net.>

In [56]:
answers.rrset[0].to_text()

'10 50 80 www.juenemann.net.'

In [62]:
answers.rrset[0].target.to_text()

'www.juenemann.net.'

In [58]:
answers.rrset[0].priority, answers.rrset[0].weight, answers.rrset[0].port

(10, 50, 80)

## Querying specific DNS server(s)
There are cases where one does not want to use alternative settings to those configured in `/etc/resolv.conf`. For this purpose one has to create and customise an instance of the `dns.resolver.Resolver` class. 
#### Example: Querying the [OpenDNS](https://www.opendns.com/) servers

In [ ]:
resolver = dns.resolver.Resolver()
resolver.nameservers = ['208.67.222.222', '208.67.220.220']
resolver.nameservers

In [ ]:
answers = resolver.query('google.com', 'NS')
list(answers.rrset)

#### Example: Disallow recursion
In this example the query flags are manipulated to disallow recursion. This will fail if the queried DNS servers are not authorative for the domain.

In [ ]:
resolver = dns.resolver.Resolver()
resolver.nameservers = ['208.67.222.222', '208.67.220.220']
resolver.nameservers

In [ ]:
resolver.set_flags(0b0000000000000000)    # Clear all flags, including 'recursive desired'
resolver.flags

In [ ]:
try:
    answers = resolver.query('www.google.com', 'A')
except Exception as e:
    print e

In [ ]:
answers = resolver.query('www.opendns.com', 'A') 
list(answers.rrset)